In [4]:
library(data.table)
library(dplyr)
library(stringr)
library(plyr)
#library("ggpubr")

#write log regression result

##use this if you dont have train.txt

#result_outrider <- fread("pred_no_na.txt")

#colnames(result_outrider)[1] <- "featureID"
#dim(result_outrider)


#measured <- read.table("prot_int.csv")

#measured[measured != 0] <- 1
#measured$featureID <- rownames(measured)


#prot_int_list <- melt(result_outrider)
#colnames(prot_int_list)[2:3] <- c("sampleID","protein_intensity")


#measured_list <- melt(measured)
#colnames(measured_list)[2:3] <- c("sampleID","measured")


#train <- merge(prot_int_list, measured_list)
#log_reg <- glm(measured~protein_intensity,family=binomial,data=train)
#train$predprob<-round(fitted(log_reg),2)
#train$missing <- 1- train$predprob

#write.table(train, "train.txt", sep = "\t", col.names = T,row.names = F, quote = F)



#read result from var_maf and train
var_maf <- read.csv("variant_with_maf_new.csv",)
outliers <- read.csv("missing_pred.csv")

print("reading done!")

[1] "reading done!"


In [6]:
var_maf 

Identifier,sampleID,frequency,dbSNP.id..g.,Gene.name..d.,Entrez.gene.id..e.,Chromosome..f.,Start..f.,End..f.,Cytoband..f.,AA.Impact..h.,X1000.Genome..i.,ESP5400..j.,SIFT.Score..k.,SNP.Type..l.,Polyphen..m.,Variant.class..n.,AF,rare
<chr>,<chr>,<dbl>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<lgl>
chr10:100010849_G_A,ME:SK-MEL-2,45.238,-,LOXL4,84171,10,100010849,100010849,10q24,R725W,0.00,0.00000,0.00,Missense,1,Type 2,1.0e-06,TRUE
chr10:100013418_T_G,ME:SK-MEL-2,56.140,rs148260647,LOXL4,84171,10,100013418,100013418,10q24,Y576S,0.00,0.00019,0.32,Missense,1,Type 1,6.0e-05,TRUE
chr10:100017453_T_G,BR:MCF7,64.151,rs1983864,LOXL4,84171,10,100017453,100017453,10q24,D405A,0.36,0.24900,0.00,Missense,1,Type 1,2.9e-01,FALSE
chr10:100017453_T_G,BR:MDA-MB-231,28.571,rs1983864,LOXL4,84171,10,100017453,100017453,10q24,D405A,0.36,0.24900,0.00,Missense,1,Type 1,2.9e-01,FALSE
chr10:100017453_T_G,BR:HS 578T,100.000,rs1983864,LOXL4,84171,10,100017453,100017453,10q24,D405A,0.36,0.24900,0.00,Missense,1,Type 1,2.9e-01,FALSE
chr10:100017453_T_G,CNS:U251,27.273,rs1983864,LOXL4,84171,10,100017453,100017453,10q24,D405A,0.36,0.24900,0.00,Missense,1,Type 1,2.9e-01,FALSE
chr10:100017453_T_G,CO:HCC-2998,47.458,rs1983864,LOXL4,84171,10,100017453,100017453,10q24,D405A,0.36,0.24900,0.00,Missense,1,Type 1,2.9e-01,FALSE
chr10:100017453_T_G,CO:HCT-116,46.000,rs1983864,LOXL4,84171,10,100017453,100017453,10q24,D405A,0.36,0.24900,0.00,Missense,1,Type 1,2.9e-01,FALSE
chr10:100017453_T_G,CO:HCT-15,55.769,rs1983864,LOXL4,84171,10,100017453,100017453,10q24,D405A,0.36,0.24900,0.00,Missense,1,Type 1,2.9e-01,FALSE


In [7]:
#get cellline name
cellline_names <- var_maf$sampleID %>%
  strsplit( ":")  %>%
  sapply( tail, 1 ) %>%
  str_replace_all("-","")%>%
  str_replace_all(" ","")%>%
  str_replace("(TB)","")%>%
  str_replace("UO31","U031")%>%
  str_replace_all("\\(|\\)", "")

var_maf$cellline_names <- cellline_names


#get celline name from outlier result
cellline_names_outliers <- sapply(strsplit(outliers$sampleID ,"_"), head, 1)

outliers$cellline_names <-cellline_names_outliers


In [8]:
head(outliers)

,featureID,sampleID,protein_intensity,measured,predprob,missing,cellline_names
,<chr>,<chr>,<dbl>,<int>,<dbl>,<dbl>,<chr>
1,A0AV96;A0AV96-2,X786O_NCI60,26.95893,1,0.97,0.03,X786O
2,A0AV96;A0AV96-2,A498_NCI60,28.67702,1,0.99,0.01,A498
3,A0AV96;A0AV96-2,A549_NCI60,29.12149,1,0.99,0.01,A549
4,A0AV96;A0AV96-2,ACHN_NCI60,29.22591,1,0.99,0.01,ACHN
5,A0AV96;A0AV96-2,BT549_NCI60,24.08905,1,0.86,0.14,BT549
6,A0AV96;A0AV96-2,CAKI1_NCI60,27.47051,1,0.98,0.02,CAKI1


In [9]:
### get gene name

#load necessary packages
library(biomaRt)
library(dplyr)


#extract most possible protein id
outliers$gene_id <-  sapply(strsplit(outliers$featureID ,";"), head, 1)

outliers$gene_id <- gsub(".*\\__", "", outliers$gene_id) 

outliers$gene_id <- gsub("\\-.*", "", outliers$gene_id)

#define list of Uniprot IDs
uniprot_ids <- unique(outliers$gene_id[!(startsWith(outliers$gene_id, "ENSEMBL")|startsWith(outliers$gene_id, "REFSEQ"))])
length(unique(uniprot_ids))

#define list of RefSeq IDs
refseq_ids <- sapply(strsplit(outliers$gene_id[startsWith(outliers$gene_id, "REFSEQ")] ,":"), tail, 1) %>% unique()

#define list of ensembl IDs
ensembl_ids <- sapply(strsplit(outliers$gene_id[startsWith(outliers$gene_id, "ENSEMBL")] ,":"), tail, 1) %>% unique()
head(uniprot_ids)

[1] 6803

[1] "A0AV96" "A0AVF1" "A0AVT1" "A0FGR8" "A0JLT2" "A0JNW5"

In [10]:
##Search in DB gene chromosom position

#use biomaRt package
mart <- useMart("ensembl", dataset = "hsapiens_gene_ensembl")

#convert Uniprot IDs to chromosome regions

uniport_results <- getBM(attributes = c("chromosome_name","start_position","end_position","uniprotswissprot"), filters = "uniprotswissprot", values = uniprot_ids, mart = mart)

#store results in a dataframe
uniport_results_df <- data.frame(uniport_results)
#nrow(uniport_results_df)
#nrow(uniport_results_df$uniprotswissprot[unique(uniport_results_df$uniprotswissprot)])

#remove chr not a number
uniport_results_df_cl <- uniport_results_df[uniport_results_df$chromosome_name %in% c(c(1:22) %>% as.character(),"X","Y"),]


#display dataframe
#print(uniport_results)



#convert refseq IDs to chromosome regions
refseq_results <- getBM(attributes = c("chromosome_name","start_position","end_position","refseq_mrna"), filters = "refseq_mrna", values = refseq_ids, mart = mart)

#store results in a dataframe
refseq_results_df <- data.frame(refseq_results)

#display dataframe (empty)
#print(refseq_results_df)



#convert ensembl IDs to chromosome regions
ensembl_results <- getBM(attributes = c("chromosome_name","start_position","end_position","ensembl_transcript_id"), filters = "ensembl_gene_id", values = ensembl_ids, mart = mart)

#store results in a dataframe
ensembl_results_df <- data.frame(ensembl_results)

#display dataframe (empty)
#print(ensembl_results_df)

#### uniport outliers

uniport_outliers <- outliers[!(startsWith(outliers$gene_id, "ENSEMBL")|startsWith(outliers$gene_id, "REFSEQ")),]

colnames(uniport_outliers)[8] <- "uniprotswissprot"
uniport_outliers_result <-join(uniport_outliers, uniport_results_df_cl, type="full",by = "uniprotswissprot")




Batch submitting query [===============>---------------]  50% eta:  9s
                                                                      



In [11]:
head(uniport_outliers_result)

,featureID,sampleID,protein_intensity,measured,predprob,missing,cellline_names,uniprotswissprot,chromosome_name,start_position,end_position
,<chr>,<chr>,<dbl>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<int>,<int>
1,A0AV96;A0AV96-2,X786O_NCI60,26.95893,1,0.97,0.03,X786O,A0AV96,4,40423267,40630875
2,A0AV96;A0AV96-2,A498_NCI60,28.67702,1,0.99,0.01,A498,A0AV96,4,40423267,40630875
3,A0AV96;A0AV96-2,A549_NCI60,29.12149,1,0.99,0.01,A549,A0AV96,4,40423267,40630875
4,A0AV96;A0AV96-2,ACHN_NCI60,29.22591,1,0.99,0.01,ACHN,A0AV96,4,40423267,40630875
5,A0AV96;A0AV96-2,BT549_NCI60,24.08905,1,0.86,0.14,BT549,A0AV96,4,40423267,40630875
6,A0AV96;A0AV96-2,CAKI1_NCI60,27.47051,1,0.98,0.02,CAKI1,A0AV96,4,40423267,40630875


In [12]:

#extract chromosom number and position from var_maf

var_maf$chromosome_name <- sapply(strsplit(var_maf$Identifier,":"), head, 1) %>% str_replace("chr","")

var_maf$position <- gsub(".*:(.+)_._.*", "\\1", var_maf$Identifier)

In [13]:
head(var_maf)

,Identifier,sampleID,frequency,dbSNP.id..g.,Gene.name..d.,Entrez.gene.id..e.,Chromosome..f.,Start..f.,End..f.,Cytoband..f.,⋯,ESP5400..j.,SIFT.Score..k.,SNP.Type..l.,Polyphen..m.,Variant.class..n.,AF,rare,cellline_names,chromosome_name,position
,<chr>,<chr>,<dbl>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<chr>,⋯,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<lgl>,<chr>,<chr>,<chr>
1,chr10:100010849_G_A,ME:SK-MEL-2,45.238,-,LOXL4,84171,10,100010849,100010849,10q24,⋯,0.00000,0.00,Missense,1,Type 2,1.0e-06,TRUE,SKMEL2,10,100010849
2,chr10:100013418_T_G,ME:SK-MEL-2,56.140,rs148260647,LOXL4,84171,10,100013418,100013418,10q24,⋯,0.00019,0.32,Missense,1,Type 1,6.0e-05,TRUE,SKMEL2,10,100013418
3,chr10:100017453_T_G,BR:MCF7,64.151,rs1983864,LOXL4,84171,10,100017453,100017453,10q24,⋯,0.24900,0.00,Missense,1,Type 1,2.9e-01,FALSE,MCF7,10,100017453
4,chr10:100017453_T_G,BR:MDA-MB-231,28.571,rs1983864,LOXL4,84171,10,100017453,100017453,10q24,⋯,0.24900,0.00,Missense,1,Type 1,2.9e-01,FALSE,MDAMB231,10,100017453
5,chr10:100017453_T_G,BR:HS 578T,100.000,rs1983864,LOXL4,84171,10,100017453,100017453,10q24,⋯,0.24900,0.00,Missense,1,Type 1,2.9e-01,FALSE,HS578T,10,100017453
6,chr10:100017453_T_G,CNS:U251,27.273,rs1983864,LOXL4,84171,10,100017453,100017453,10q24,⋯,0.24900,0.00,Missense,1,Type 1,2.9e-01,FALSE,U251,10,100017453


In [14]:
#map chromosomes

match_gene_id <- function(A, B) {
  # Add new column to B dataframe to store the matched gene ID
  B$uniprotswissprot <- NA
  #B$uniprotswissprot_2 <- NA
  
  # Loop through each row in B
  for (i in 1:nrow(B)) {
    # Subset A to only include rows with the same chromosome name as B
    A_subset <- subset(A, A$chromosome_name == B$chromosome_name[i])
    #Check if the position of SNP is between start and end position
    A_subset <- A_subset[A_subset$start_position <= B$position[i] & A_subset$end_position >= B$position[i],]
    #if there is a match assign the gene_id to the B dataframe
    if(nrow(A_subset) > 0){
      #print(A_subset$uniprotswissprot)
      B$uniprotswissprot[i] <- A_subset$uniprotswissprot[1]
      #if(length(A_subset$uniprotswissprot) > 1){
      #    B$uniprotswissprot_2[i] <- A_subset$uniprotswissprot[2]
      #  }
      
    }else{
      B$uniprotswissprot[i] <- "None_found"
    }
    
  }
  return(B)
}

#mapping start
#test <-  match_gene_id(uniport_results_df_cl,var_maf)

var_maf_with_gene <- match_gene_id(uniport_results_df_cl,var_maf)

var_maf_with_gene_cl <- var_maf_with_gene[var_maf_with_gene$uniprotswissprot != "None_found",]

var_maf_outlier_result <-join(uniport_outliers, var_maf_with_gene_cl,by = c("uniprotswissprot","cellline_names"))

var_maf_outlier_result_cl <- na.omit(var_maf_outlier_result)


#write.table(var_maf_outlier_result_cl, "outlier_prob_with_maf.txt",quote = F,row.names = F,col.names = T,sep = ",")


In [15]:
head(var_maf_outlier_result_cl)

,featureID,sampleID,protein_intensity,measured,predprob,missing,cellline_names,uniprotswissprot,Identifier,sampleID.1,⋯,X1000.Genome..i.,ESP5400..j.,SIFT.Score..k.,SNP.Type..l.,Polyphen..m.,Variant.class..n.,AF,rare,chromosome_name,position
,<chr>,<chr>,<dbl>,<int>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<lgl>,<chr>,<chr>
2,A0AV96;A0AV96-2,A498_NCI60,28.67702,1,0.99,0.01,A498,A0AV96,chr4:40428010_T_C,RE:A498,⋯,0.65,0.745,0.88,Missense,0.318,Type 1,0.27,FALSE,4,40428010
4,A0AV96;A0AV96-2,ACHN_NCI60,29.22591,1,0.99,0.01,ACHN,A0AV96,chr4:40428010_T_C,RE:ACHN,⋯,0.65,0.745,0.88,Missense,0.318,Type 1,0.27,FALSE,4,40428010
5,A0AV96;A0AV96-2,BT549_NCI60,24.08905,1,0.86,0.14,BT549,A0AV96,chr4:40428010_T_C,BR:BT-549,⋯,0.65,0.745,0.88,Missense,0.318,Type 1,0.27,FALSE,4,40428010
7,A0AV96;A0AV96-2,CCRFCEM_NCI60,22.85227,1,0.74,0.26,CCRFCEM,A0AV96,chr4:40428010_T_C,LE:CCRF-CEM,⋯,0.65,0.745,0.88,Missense,0.318,Type 1,0.27,FALSE,4,40428010
8,A0AV96;A0AV96-2,COLO205_NCI60,32.45874,1,1.00,0.00,COLO205,A0AV96,chr4:40428010_T_C,CO:COLO 205,⋯,0.65,0.745,0.88,Missense,0.318,Type 1,0.27,FALSE,4,40428010
9,A0AV96;A0AV96-2,DU145_NCI60,28.00763,1,0.99,0.01,DU145,A0AV96,chr4:40428010_T_C,PR:DU-145,⋯,0.65,0.745,0.88,Missense,0.318,Type 1,0.27,FALSE,4,40428010


In [16]:
cor(var_maf_outlier_result_cl$missing, var_maf_outlier_result_cl$AF,  method = "pearson", use = "complete.obs")

[1] -0.01640072

In [17]:
ggplot(var_maf_outlier_result_cl[var_maf_outlier_result_cl$cellline_names =="A498",], aes(missing %>% as.numeric(),AF %>% as.numeric()))+
  geom_point()

ERROR: Error in ggplot(var_maf_outlier_result_cl[var_maf_outlier_result_cl$cellline_names == : konnte Funktion "ggplot" nicht finden
